# Overview of MODIS Near Real-Time (NRT) Data Processing Notebook

## Introduction

This notebook demonstrates the process of accessing, downloading, and processing MODIS (Moderate Resolution Imaging Spectroradiometer) Near Real-Time (NRT) data for environmental monitoring and analysis using the mcimageprocessing package.

## Objective

The main objective of this notebook is to provide a practical example of how to programmatically:
1. Set up environment configurations for data processing.
2. Load and manipulate geospatial boundaries using GeoPandas.
3. Interact with Google Earth Engine through an API to fetch MODIS NRT data.
4. Download, process, and clip satellite imagery data to focus on specific areas of interest.
5. Prepare the data for further analysis or visualization tasks.

## Workflow

The workflow of the notebook is structured as follows:
- **Setting Up Environment Variables**: Configure the paths necessary for the APIs and processing tools.
- **Importing Modules**: Load all necessary Python libraries and custom modules required for processing.
- **Loading Geospatial Data**: Read the area of interest defined in a GeoJSON file.
- **Initializing Data Managers**: Set up connections to data sources like Google Earth Engine.
- **Retrieving and Processing Data**:
  - Fetch available MODIS NRT data dates.
  - Determine the spatial tiles that intersect with the area of interest.
  - List and download the relevant MODIS NRT data files.
  - Process the downloaded data into a usable format (e.g., TIFF).
- **Post-Processing**:
  - Merge processed files to create a cohesive dataset.
  - Clip the dataset to the boundaries of the specified area of interest.

### NRT_DATA_LAYER OPTIONS: 
1. 'Water Counts 1-Day 250m Grid_Water_Composite'
2. 'Water Counts CS 1-Day 250m Grid_Water_Composite'
3. 'Valid Counts 1-Day 250m Grid_Water_Composite'
4. 'Valid Counts CS 1-Day 250m Grid_Water_Composite'
5. 'Flood 1-Day 250m Grid_Water_Composite'
6. 'Flood 1-Day CS 250m Grid_Water_Composite'
7. 'Water Counts 2-Day 250m Grid_Water_Composite'
8. 'Valid Counts 2-Day 250m Grid_Water_Composite'
9. 'Flood 2-Day 250m Grid_Water_Composite'
10. 'Water Counts 3-Day 250m Grid_Water_Composite'
11. 'Valid Counts 3-Day 250m Grid_Water_Composite'
12. 'Flood 3-Day 250m Grid_Water_Composite'


## Step 1: Setting Up Environment Variables

### Set the required environment variable for configuration directory. Alternatively, if not set, when mcimageprocessing is imported, it will request the user define a path and subsequently set the path to the config file.

In [ ]:
import os

# Set environment variable
os.environ['CONFIG_DIR'] = # SET PATH TO CONFIG DIRECTORY

folder_path = # SET FOLDER PATH FOR DOWNLOADS/PROCESSING

NRT_DATA_LAYER = # SET LAYER NAME FOR NRT DATA. EXAMPLE USING Flood 3-Day 250m Grid_Water_Composite

## Step 2: Importing Necessary Modules

### Import the modules needed for processing MODIS NRT data.

In [ ]:
from mcimageprocessing.programmatic.APIs.EarthEngine import EarthEngineManager
from mcimageprocessing.programmatic.APIs.ModisNRT import ModisNRT
from mcimageprocessing.jupyter.JupyterAPI import JupyterAPI
from mcimageprocessing.programmatic.shared_functions.utilities import generate_bbox, clip_raster
import json
import geopandas as gpd

## Step 3: Loading Geospatial Data

### Load geojson geometry to define the area of interest.

In [ ]:
with open('geometry.geojson', 'r') as fp:
    geometry_json = json.load(fp)
gdf = gpd.GeoDataFrame.from_features(geometry_json['features'])

## Step 4: Initializing MODIS NRT Data Manager

### Initialize the MODIS Near Real-Time (NRT) data manager with Earth Engine.

In [ ]:
modis = ModisNRT(ee_manager=EarthEngineManager())

## Step 5: Retrieving Available MODIS Dates

### Retrieve available dates for MODIS NRT data.

In [ ]:
available_dates = modis.get_modis_nrt_dates()

## Step 6: Generating Bounding Box

### Generate a bounding box from the GeoDataFrame.

In [ ]:
bbox = generate_bbox(gdf)

## Step 7: Retrieving MODIS Tiles

### Get the MODIS tiles that intersect with the bounding box.

In [ ]:
tiles = modis.get_modis_tile(bbox)

## Step 8: Listing MODIS Files for Download

### List the MODIS NRT files available for download for the specified date and tiles.

In [ ]:
files_for_download = modis.get_modis_nrt_file_list(tiles=tiles, modis_nrt_params={'date': available_dates[0]})

## Step 9: Downloading and Processing MODIS Files

### Download and process the MODIS NRT files, converting them to a specified format.

In [ ]:
tif_list = []

for file in files_for_download:
    modis.download_and_process_modis_nrt(url=file, folder_path=folder_path, 
                                   hdf_files_to_process=files_for_download, subdataset=NRT_DATA_LAYER,
                                        tif_list=tif_list)

## Step 10: Merging TIFF Files

### Merge the processed TIFF files into a single file.

In [ ]:
merged_file = modis.merge_files(tif_list=tif_list, folder=folder_path)

## Step 11: Clipping the Raster File

### Clip the merged raster file to the boundary of the GeoDataFrame.

In [ ]:
merged_file_path = clip_raster(merged_file, gdf)

## Step 11: Calculate Population in Flood Area

### Use the calculate_population_in_flood_area function to calculate the total population in flood area.

- **Population Data Type Options**: WorldPop, GPWv4
- **Year**: years available include yearly from 2000 to 2020 for WorldPop, and 2000, 2005, 2010, 2015, and 2020 for GPWv4.
- **Population Data Source Options**: 'Residential Population', 'Age and Sex Structures' for WorldPop, and 'Population Count' 'Population Density', and 'UN-Adjusted Population Count' for GPWv4.

In [ ]:
modis.calculate_population_in_flood_area(merged_file_path, year=2020, population_data_type='WorldPop', population_data_source='Residential Population',
                                        folder_output=folder_path)